In [ ]:
%pip install wandb

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  # For splitting the dataset
from sklearn.ensemble import RandomForestRegressor  # For the Random Forest regression model
import shap
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
import os
import wandb
import re

pd.set_option('display.expand_frame_repr', False)


In [ ]:
def save_dataframe(df,file_name):
  from google.colab import drive
  drive.mount('/content/drive')
  file_path = "/content/drive/My Drive/"+file_name+".csv"
  df.to_csv(file_path, index=False)
  print(f"File saved successfully at: {file_path}")

def load_dataframe_from_drive(file_name='Train'):
    """
    Function to load a DataFrame from a Google Drive file path.
    """
    from google.colab import drive
    drive.mount('/content/drive')

    # Load the file from Google Drive
    file_path = "/content/drive/My Drive/"+file_name+".csv"  # Replace with the saved file path
    df = pd.read_csv(file_path)
    return df

def save_model(model,model_name):
  from google.colab import drive
  drive.mount('/content/drive')

  import joblib

  # Assuming 'rf_model' is your trained RandomForestRegressor
  model_path = '/content/drive/My Drive/'+model_name+'.pkl'
  joblib.dump(model, model_path)

  print(f"Model saved at {model_path}")

def load_model(model_name):
  import joblib

  model = joblib.load('/content/drive/My Drive/'+model_name+'.pkl')
  print("Model loaded successfully!")
  return model

def RMSLE(y_test, y_pred):
    '''
    RSMLE approximates the percent change
    '''
    return np.sqrt(np.mean((np.log(y_pred) - np.log(y_test))**2))

def RMSE(y_, y_pred_):
    '''
    RSME
    '''
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

def train_model(df,column_to_predict,test_size_value=0.3,random_state_value=42):
    '''
    Function to train a Random Forest regression model on a given DataFrame.
    '''
    print("train_model START")

    !wandb login --relogin

    # Initialize a new W&B run
    wandb.init(
        project="Predict-heavy-machinery-price"
    )

    X = df.drop(columns=[column_to_predict])
    y = df[column_to_predict]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_value, random_state=random_state_value)
    model = RandomForestRegressor()
    model.fit(X_train, y_train)


    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_test_pred)

    wandb.log({"mean_squared_error": mse})
    wandb.log({"Train RMSE": RMSE(y_train, y_train_pred)})
    wandb.log({"Test RMSE": RMSE(y_test, y_test_pred)})

    # Optional: Log feature importances
    importances = model.feature_importances_

    # Create a zip of feature names and feature importances
    feature_zip = zip(model.feature_names_in_, model.feature_importances_)

    # Sort the zip by feature importance in descending order
    sorted_feature_zip = sorted(feature_zip, key=lambda x: x[1], reverse=True)

    # Print the sorted feature names, importances, and index
    for idx, (feat, importance) in enumerate(sorted_feature_zip, 1):
        wandb.log({f"feature_{idx}_{feat}_importance" : importance})

    # Finish the run
    wandb.finish()

    print("RMSE Baseline accuracy:", y_test.std())
    print("Train RMSE:", RMSE(y_train, y_train_pred))
    print("Test RMSE:", RMSE(y_test, y_test_pred))

    print("train_model END")

    return model,(X_train, X_test, y_train, y_test)



def generate_X_Y_params(df,column_to_predict,test_size_value=0.3,random_state_value=42):
    X = df.drop(columns=[column_to_predict])
    y = df[column_to_predict]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_value, random_state=random_state_value)
    return X_train, X_test, y_train, y_test

def list_unique_values(df, column_name):
    """
    Function to list all unique values in a given column of a DataFrame.

    Parameters:
    - df: pandas DataFrame
    - column_name: Name of the column to find unique values

    Returns:
    - A list of unique values in the specified column.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")

    unique_values = df[column_name].unique()
    return unique_values


# Function to calculate YearMade based on ModelID
def calc_YearMade(df, model_id):
    valid_years = df.loc[(df['ModelID'] == model_id) & (df['YearMade'] > 1000), 'YearMade']
    return int(valid_years.mean()) if not valid_years.empty else 1000  # Default if no valid years exist

def update_YearMade(df):
  print("update_YearMade START")
  # Update YearMade where it is 1000
  df.loc[df['YearMade'] == 1000, 'YearMade'] = df.loc[df['YearMade'] == 1000, 'ModelID'].apply(lambda model_id: calc_YearMade(df, model_id))
  # Compute the mean excluding rows where YearMade == 1000
  mean_yearmade = df.loc[df['YearMade'] != 1000, 'YearMade'].mean()

  # Update all rows where YearMade == 1000 with the calculated mean
  df.loc[df['YearMade'] == 1000, 'YearMade'] = int(round(mean_yearmade))

  print("update_YearMade END")

  return df


def update_auctioneerID(df):
  print("update_auctioneerID START")

  # Assuming 'df' is your DataFrame
  # Fill null values in the 'auctioneerID' column with 100.0
  if 'auctioneerID' in df.columns:
      df['auctioneerID'] = df['auctioneerID'].fillna(100.0)
      #print("'auctioneerID' null values filled with 100.0 successfully!")
  #else:
  #    print("Column 'auctioneerID' does not exist in the DataFrame.")
  print("update_auctioneerID END")

  return df


def convert_date_columns(df, date_column):
    """
    Convert date column into multiple numeric features like year, month, day, etc.
    Handles potential issues like missing date_column or invalid data.
    """
    print("convert_date_columns START")

    # Ensure the date_column exists in the DataFrame
    if date_column not in df.columns:
        raise KeyError(f"Column '{date_column}' does not exist in the DataFrame.")

    # Convert the column to datetime
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')  # Handle invalid date strings

    # Check if any date values are invalid after conversion
    if df[date_column].isnull().all():
        raise ValueError(f"All values in '{date_column}' could not be converted to datetime.")

    # Extract date features
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['day'] = df[date_column].dt.day
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['is_weekend'] = df['day_of_week'] >= 5  # Saturday=5, Sunday=6

    # Drop the original date column
    df = df.drop(columns=[date_column])

    print("convert_date_columns END")

    return df

def update_MachineHoursCurrentMeter(df):
  print("update_MachineHoursCurrentMeter START")

  # Update the 'MachineHoursCurrentMeter' column: replace NaN or 0 with 0
  column_name = 'MachineHoursCurrentMeter'

  # Check if the column exists
  if column_name in df.columns:
      df[column_name] = df[column_name].fillna(0)  # Replace NaN with 0
      df[column_name] = df[column_name].replace(0, 0)  # Ensure 0 stays 0

  print("update_MachineHoursCurrentMeter END")

  return df

def encode_and_impute(df):
    '''
    Function to convert categorical columns into category codes
    and impute missing values in numerical columns.
    '''
    print("Data Preprocessing START")

    # Handling categorical variables
    for col in df.select_dtypes(['object']):
        df[col] = df[col].fillna("None or Unspecified").astype('category')

    for col in df.select_dtypes(['category']):
        df[col] = df[col].cat.codes  # Convert categories to numerical codes

    # Handling numerical missing values
    for col in df.select_dtypes(['number']):
        df[col] = df[col].fillna(df[col].median())  # Fill NaNs with median

    print("Data Preprocessing END")
    return df

def update_col_NaN(df,column_name,update_to):
   df[column_name]=df[column_name].fillna(update_to)
   return df

def clean_column(df, column_name):
    """
    Cleans a column in the DataFrame.
    - Replaces 'None or Unspecified' with 1.0.
    - Removes double quotes (") from values.
    - Removes the word "inch" from values.
    - Removes all spaces from values.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The column to clean.

    Returns:
        pd.DataFrame: Updated DataFrame with cleaned column.
    """
    if column_name not in df.columns:
        raise KeyError(f"Column '{column_name}' not found in DataFrame.")

    df[column_name] = df[column_name].fillna(0.0)

    df[column_name] = df[column_name].replace('None or Unspecified', '0.0')

    # Function to clean values
    def clean_value(value):
        str_value = str(value)
        str_value = str_value.replace('"', '')  # Remove double quotes
        str_value = str_value.replace('inch', '')  # Remove 'inch'
        str_value = str_value.replace(' ', '')  # Remove spaces
        return str_value

    df[column_name] = df[column_name].apply(clean_value)
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')

    # Replace 0.0 values with the column median
    median_value = df[df[column_name] != 0][column_name].median()
    df[column_name] = df[column_name].replace(0.0, median_value)

    return df

def convert_feet_inches(df, column_name):
    """
    Converts a column containing feet and inches strings to numeric inches.
    - Replaces 'None or Unspecified' with "0' 0".
    - Converts values like "9' 6"" (feet and inches) into total inches.
    - Converts the column data type to numeric.
    - Replaces 0 values with the column median.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The column to process.

    Returns:
        pd.DataFrame: Updated DataFrame with cleaned numeric column.
    """
    if column_name not in df.columns:
        raise KeyError(f"Column '{column_name}' not found in DataFrame.")

    # Replace 'None or Unspecified' with "0' 0" before processing
    df[column_name] = df[column_name].replace('None or Unspecified', "0' 0")

    # Function to convert feet and inches format to numeric inches
    def convert_to_inches(value):
        str_value = str(value).strip()

        # Match feet and inches pattern
        match = re.match(r"(\d+)'\s*(\d+)?\"?", str_value)
        if match:
            feet = int(match.group(1))
            inches = int(match.group(2)) if match.group(2) else 0
            return feet * 12 + inches

        return 0  # If format is invalid, default to 0

    df[column_name] = df[column_name].apply(convert_to_inches)

    # Convert column to numeric
    df[column_name] = pd.to_numeric(df[column_name], errors='coerce')

    # Replace 0 values with the column median
    median_value = df[df[column_name] != 0][column_name].median()
    df[column_name] = df[column_name].replace(0, median_value)

    return df

def process_fidesc_secondary_series(df):
    """
    Iterates over rows and updates fiSecondaryDesc and fiModelSeries based on fiModelDesc and fiBaseModel.

    Parameters:
    df (DataFrame): The input DataFrame.

    Returns:
    DataFrame: The modified DataFrame with updated fiSecondaryDesc and fiModelSeries.
    """
    # Ensure all values are treated as strings to prevent TypeError
    df['fiSecondaryDesc'] = df['fiSecondaryDesc'].astype(str)
    df['fiModelSeries'] = df['fiModelSeries'].astype(str)

    # Get unique values in fiSecondaryDesc (excluding 'nan')
    fiSecondaryDesc_unique_values = sorted(
        [val for val in df['fiSecondaryDesc'].unique() if val.lower() != 'nan'],
        key=len, reverse=True
    )

    for index in df.index:
        fi_model_desc = str(df.at[index, 'fiModelDesc'])  # Convert to string
        fi_base_model = str(df.at[index, 'fiBaseModel'])  # Convert to string
        fi_secondary_desc = df.at[index, 'fiSecondaryDesc']
        fi_model_series = df.at[index, 'fiModelSeries']

        # Create fidesc by removing fiBaseModel from fiModelDesc
        fidesc = fi_model_desc.replace(fi_base_model, '', 1).strip()

        # Skip empty or NaN fidesc
        if fidesc.lower() == 'nan' or fidesc == '':
            continue

        # Update fiSecondaryDesc if fidesc starts with a known unique value
        if fi_secondary_desc.lower() == 'nan':  # Only update missing values
            for value in fiSecondaryDesc_unique_values:
                if fidesc.startswith(value):  # Ensure the match is at the start
                    df.at[index, 'fiSecondaryDesc'] = value  # Assign matching unique value
                    #fidesc = fidesc[len(value):].strip()  # **Remove the exact match**
                    fidesc = fidesc.replace(value,'')
                    break  # Stop after the first match

        # Ensure fidesc is fully cleaned before updating fiModelSeries
        fidesc = fidesc.strip()

        # Prevent fiModelSeries from receiving fiSecondaryDesc value again
        if fi_model_series.lower() == 'nan' and fidesc and fidesc != df.at[index, 'fiSecondaryDesc']:
            # If fiModelSeries still contains fiSecondaryDesc, remove it
            if fidesc.startswith(df.at[index, 'fiSecondaryDesc']):
                fidesc = fidesc[len(df.at[index, 'fiSecondaryDesc']):].strip()

            df.at[index, 'fiModelSeries'] = fidesc  # Assign cleaned fidesc

        #print(f"{fi_model_desc},{fi_base_model},fiSecondaryDesc={df.at[index, 'fiSecondaryDesc']},fiModelSeries={df.at[index, 'fiModelSeries']}")

    return df

def group_low_frequency_categories(df, threshold_percent=1.0):
    """
    Groups low-frequency categories in all categorical columns based on a dynamic threshold.

    Parameters:
    df (DataFrame): The input DataFrame.
    threshold_percent (float): The percentage (default 1%) of total non-null values below which categories are grouped as "Other".

    Returns:
    DataFrame: The updated DataFrame with grouped low-frequency categories.
    """
    df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for col in df.select_dtypes(include=['object']).columns:  # Only process categorical columns
        value_counts = df[col].value_counts()  # Get frequency of each category

        threshold = df[col].count() * (threshold_percent / 100)  # Dynamic threshold
        low_freq_categories = value_counts[value_counts < threshold].index  # Find categories below threshold

        if len(low_freq_categories) > 0:  # Only modify if there are low-frequency categories
            df[col] = df[col].apply(lambda x: 'Other' if x in low_freq_categories else x)
            print(f"Updated column: {col}, grouped {len(low_freq_categories)} categories into 'Other'")

    return df


def prepare_dataframe(df_pre:pd.DataFrame):
  '''04/02/2025 Start'''
  df_pre=update_YearMade(df_pre)
  print()

  df_pre=update_MachineHoursCurrentMeter(df_pre)
  print()

  df_pre=update_auctioneerID(df_pre)
  print()

  #df_pre=update_Enclosure(df_pre)
  #print()

  df_pre=convert_date_columns(df_pre,'saledate')
  print()

  #df_pre=encode_all_categories(df_pre)
  '''04/02/2025 End'''

  '''Start changes 06-02-2025'''
  df_pre = process_fidesc_secondary_series(df_pre)

  #df_pre=update_col_NaN(df_pre,'UsageBand','None or Unspecified')
  #df_pre=convert_feet_inches(df_pre,'Stick_Length')
  #df_pre=clean_column(df_pre,'Tire_Size')
  df_pre=clean_column(df_pre,'Undercarriage_Pad_Width')
  '''End changes 06-02-2025'''

  '''Start changes 06-02-2025 0015'''

  df_pre=update_col_NaN(df_pre,'ProductSize','None or Unspecified')
  df_pre=update_col_NaN(df_pre,'Drive_System','No')
  df_pre=update_col_NaN(df_pre,'Stick','None or Unspecified')
  df_pre=update_col_NaN(df_pre,'Engine_Horsepower','No')
  df_pre=update_col_NaN(df_pre,'Track_Type','None or Unspecified')
  df_pre=update_col_NaN(df_pre,'Grouser_Type','None or Unspecified')
  df_pre=update_col_NaN(df_pre,'Differential_Type','None or Unspecified')
  df_pre=update_col_NaN(df_pre,'Steering_Controls','No')
  '''End changes 06-02-2025 0015'''

  '''06022025 1620 Grouping Start'''
  df_pre = group_low_frequency_categories(df_pre)
  '''06022025 1620 Grouping End'''

  '''Start changes 05-02-2025 1530'''
  df_pre=encode_and_impute(df_pre)
  '''End changes 05-02-2025 1530'''

  return df_pre

def run_test(model):
  file_path = "/content/drive/My Drive/Valid.csv"  # Replace with the saved file path
  df_valid = pd.read_csv(file_path)
  df_valid['SalePrice']=None
  df_valid=prepare_dataframe(df_valid)
  X = df_valid.drop(columns=['SalePrice'])
  y = df_valid['SalePrice']
  y = model.predict(X)

  df_valid.loc[X.index, 'SalePrice'] = y

  file_path = "/content/drive/My Drive/output.csv"  # Save to Google Drive root
  df_valid[['SalesID', 'SalePrice']].to_csv(file_path, index=False)

  print(f"File saved successfully at: {file_path}")

def run_train():
  pd.set_option('display.expand_frame_repr', False)

  df=load_dataframe_from_drive()
  #df = df.sample(n=10000, random_state=42)
  df=prepare_dataframe(df)
  model,xs=train_model(df=df,column_to_predict='SalePrice')
  #save_dataframe(df,'04022025_df')
  #save_model(model,'04022025_model')
  #save_dataframe(df,'05022025_df')
  #save_model(model,'05022025_model')
  #save_dataframe(df,'06022025_df')
  #save_model(model,'06022025_model')
  #save_dataframe(df,'06022025_1440_df')
  #save_model(model,'06022025_1440_model')

  '''
  df=load_dataframe_from_drive('04022025_df')
  model=load_model('04022025_model')
  xs=generate_X_Y_params(df,'SalePrice')
  '''
  return model,xs


**RUN CELL**

In [ ]:
print('Starting.....')
model,xs=run_train()
print('Testing...')
run_test(model)


Starting.....
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-18-6e4d201a662a>:17: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


update_YearMade START
update_YearMade END

update_NaN_To_None_or_Unspecified START
update_NaN_To_None_or_Unspecified END

update_MachineHoursCurrentMeter START
update_MachineHoursCurrentMeter END

update_auctioneerID START
update_auctioneerID END

convert_date_columns START
convert_date_columns END

Updated column: fiModelDesc, grouped 4994 categories into 'Other'
Updated column: fiBaseModel, grouped 1932 categories into 'Other'
Updated column: fiSecondaryDesc, grouped 161 categories into 'Other'
Updated column: fiModelSeries, grouped 191 categories into 'Other'
Updated column: fiModelDescriptor, grouped 128 categories into 'Other'
Updated column: fiProductClassDesc, grouped 35 categories into 'Other'
Updated column: state, grouped 27 categories into 'Other'
Updated column: Drive_System, grouped 1 categories into 'Other'
Updated column: Enclosure, grouped 3 categories into 'Other'
Updated column: Pad_Type, grouped 2 categories into 'Other'
Updated column: Transmission, grouped 4 catego

Test RMSE,▁
Train RMSE,▁
feature_10_ProductGroupDesc_importance,▁
feature_11_ProductGroup_importance,▁
feature_12_MachineID_importance,▁
feature_13_month_importance,▁
feature_14_day_importance,▁
feature_15_Enclosure_importance,▁
feature_16_state_importance,▁
feature_17_Pushblock_importance,▁
feature_18_fiModelDescriptor_importance,▁


RMSE Baseline accuracy: 22932.4005340408
Train RMSE: 2660.443512828269
Test RMSE: 7105.531836041139
train_model END
Testing...
update_YearMade START
update_YearMade END

update_NaN_To_None_or_Unspecified START
update_NaN_To_None_or_Unspecified END

update_MachineHoursCurrentMeter START
update_MachineHoursCurrentMeter END

update_auctioneerID START
update_auctioneerID END

convert_date_columns START
convert_date_columns END

Updated column: fiModelDesc, grouped 1729 categories into 'Other'
Updated column: fiBaseModel, grouped 767 categories into 'Other'
Updated column: fiSecondaryDesc, grouped 67 categories into 'Other'
Updated column: fiModelSeries, grouped 96 categories into 'Other'
Updated column: fiModelDescriptor, grouped 61 categories into 'Other'
Updated column: fiProductClassDesc, grouped 26 categories into 'Other'
Updated column: state, grouped 22 categories into 'Other'
Updated column: Drive_System, grouped 1 categories into 'Other'
Updated column: Enclosure, grouped 1 categor

<ipython-input-18-6e4d201a662a>:485: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[56140.   76575.   32952.75 ... 10777.5  13177.5  26462.5 ]' has dtype incompatible with int8, please explicitly cast to a compatible dtype first.
  df_valid.loc[X.index, 'SalePrice'] = y
